<a href="https://colab.research.google.com/github/bred91/Computational_Intelligence_2022-2023/blob/main/Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Raffaele Pane `<raffaele.pane@studenti.polito.it>`  
[`https://github.com/bred91/Computational_Intelligence_2022-2023/tree/main/lab1`](https://github.com/bred91/Computational_Intelligence_2022-2023/tree/main/lab1) 

# Lab 1: Set Covering

First lab + peer review. List this activity in your final report, it will be part of your exam.

## Task

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$, 
determine, if possible, $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$
such that each number between $0$ and $N-1$ appears in at least one list

$$\forall n \in [0, N-1] \ \exists i : n \in L_{s_i}$$

and that the total numbers of elements in all $L_{s_i}$ is minimum. 

## Instructions

* Create the directory `lab1` inside the course repo (the one you registered with Andrea)
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)
* Use `problem` to generate the problems with different $N$
* In the `README.md`, report the the total numbers of elements in $L_{s_i}$ for problem with $N \in [5, 10, 20, 100, 500, 1000]$ and the total number on $nodes$ visited during the search. Use `seed=42`.
* Use `GitHub Issues` to peer review others' lab

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.

**Deadline**

* Sunday, October 16th 23:59:59 for the working solution
* Sunday, October 23rd 23:59:59 for the peer reviews

## Solution

In [1]:
# import
import random
import numpy as np
import heapq
from typing import Callable
import logging
import itertools

In [2]:
class PriorityQueue:
    """A basic Priority Queue with simple performance optimizations"""

    def __init__(self):
        self._data_heap = list()
        self._data_set = set()

    def __bool__(self):
        return bool(self._data_set)

    def __contains__(self, item):
        return item in self._data_set

    def push(self, item, p=None):
        assert item not in self, f"Duplicated element"
        if p is None:
            p = len(self._data_set)
        self._data_set.add(item)
        heapq.heappush(self._data_heap, (p, item))

    def pop(self):
        p, item = heapq.heappop(self._data_heap)
        self._data_set.remove(item)
        return item

In [3]:
def problem(N, seed=None):
    """Generate a random problem instance"""
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [4]:
logging.getLogger().setLevel(logging.INFO)

In [5]:
def h1(state,N):
    '''Heuristic function: number of elements in the goal state that are not in the current state'''
    logging.debug(N - len(state))
    return N - len(state)

In [6]:
def h2(state,N,inter):
    '''Heuristic function: number of elements in the goal state that are not in the current state'''
    logging.debug(N - len(state))
    return (N - len(state)) if inter == 0 else (N - len(state)) * (0.75 + inter)

### A*

In [7]:
def a_star_search(
    N : int
):
    frontier = PriorityQueue()
    parent_state = dict()
    state_cost = dict()  
    steps = dict()  
    priority_function = lambda state,N: state_cost[state] + h1(state,N) 
    #priority_function = lambda state,N,inter: state_cost[state] + h2(state,N,inter)

    goal = set(range(N))
    sorted_all_list = sorted(problem(N, seed=42), key=lambda l: len(l))
    all_lists = list(sorted_all_list for sorted_all_list,_ in itertools.groupby(sorted_all_list)) # remove duplicates

    state = frozenset()
    parent_state[state] = None
    state_cost[state] = 0
    steps[state] = []
    

    while state is not None and goal != state:
        for a_list in all_lists:
            new_state = state 
            new_state |= set(a_list)            
            cost = len(a_list)
            inter = state.intersection(set(a_list))
            logging.debug(f"a: {a_list} , state: {state} , new_state: {new_state} , intersection: {inter}")

            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                steps[new_state] = {new_state: a_list } 
                frontier.push(new_state, p = priority_function(new_state,N,))
                #frontier.push(new_state, p = priority_function(new_state,N,len(inter)))
                logging.debug(f"Added new node {a_list} to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                steps[new_state] = {new_state: a_list } 
                logging.debug(f"Updated node {a_list} cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list() 
    path_steps = list()   
    s = state
    
    while s:
        path.append(list(s))
        path_steps.append(list(steps[s].values())[0])
        logging.debug(f"Path: {path}, State: {s}, Steps: {path_steps}")        
        s = parent_state[s]
    
    logging.info(f"A* solution for N={N}: w={sum(len(_) for _ in path_steps)} (bloat={(sum(len(_) for _ in path_steps)-N)/N*100:.0f}%)\n"+
    f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")

    logging.debug(f"Steps: \n{list(reversed(path_steps))}")
    logging.debug(f"States: \n{list(reversed(path))}")

For higher N, it explodes.

In [8]:
for N in [5,10,20,30]:
    a_star_search(N)

INFO:root:A* solution for N=5: w=5 (bloat=0%)
Found a solution in 3 steps; visited 32 states
INFO:root:A* solution for N=10: w=10 (bloat=0%)
Found a solution in 3 steps; visited 692 states
INFO:root:A* solution for N=20: w=23 (bloat=15%)
Found a solution in 5 steps; visited 15,595 states
INFO:root:A* solution for N=30: w=36 (bloat=20%)
Found a solution in 5 steps; visited 3,121,974 states
